In [ ]:
import pandas as pd
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
import numpy as np
import os

In [ ]:
def get_key(indexes):
  op=["OPERATING", "OPERATIONS", "OPERATIONAL"]
  loss=["INCOME", "LOSS", "PROFIT", "(LOSS)", "(INCOME)", "INCOME/(LOSS)", "EARNINGS", "EARNING"]

  temp=0
  for i in indexes:
    i=str(i)
    j=i.split()
    l1=[k for k in j if k in op]
    l2=[k for k in j if k in loss]
    if len(l1)!=0 and len(l2)!=0:
      temp=" ".join(j)
      break
  
  return temp

ebitda_change=pd.DataFrame(columns=["2021-2020", "2020-2019", "2019-2018", "2018-2017", "2017-2016"])

path="/content/drive/MyDrive/InterIIT/Data"
company_list=os.listdir(path)

company_list.remove("SOLARWINDS CORP")
company_list.remove("CITRIX SYSTEMS INC")
bad_companies={"SOLARWINDS CORP", "CITRIX SYSTEMS INC"}



for i in range(len(company_list)):

  try:

    print(company_list[i])

    scale1="normal"

    sheets="/content/drive/MyDrive/InterIIT/Data/"+company_list[i]+"/10-K"

    try:
      lst=os.listdir(sheets)
    except Exception as e:
      print(e, f"related to company {company_list[i]}")
      ebitda_change.loc[company_list[i]]=np.nan
      continue
    
  
    # If a company has no data i.e no 10-K files are present.
    if len(lst)==0:
      ebitda_change.loc[company_list[i]]=np.nan
      continue

    
    for path in lst:
                
      file_path="/content/drive/MyDrive/InterIIT/Data/"+company_list[i]+"/10-K/"+path
      possible_sheet_names = ['Consolidated Statements of Oper','CONSOLIDATED STATEMENTS OF OPER',
                              "CONSOLIDATED_STATEMENTS_OF_OP",'Consolidated Statement of Opera','Statements of Operations',
                              'Consolidated Statements of Inco','CONSOLIDATED STATEMENTS OF INCO','INCOME STATEMENTS','INCOME_STATEMENTS',
                              'CONSOLIDATED AND COMBINED STATE','Condensed Consolidated Statemen','Consolidated Statements of Comp']
      found=False
      try:
            sheet1 = pd.read_excel(file_path,sheet_name=possible_sheet_names[0],index_col=0)
            found=True
      except:
            try:
                sheet1 = pd.read_excel(file_path,sheet_name=possible_sheet_names[1],index_col=0)
                found=True
            except:
                try:
                    sheet1 = pd.read_excel(file_path,sheet_name=possible_sheet_names[2],index_col=0)
                    found=True
                except:
                    try:
                        sheet1 = pd.read_excel(file_path,sheet_name=possible_sheet_names[3],index_col=0)
                        found=True
                    except:
                        try:
                            sheet1 = pd.read_excel(file_path,sheet_name=possible_sheet_names[4],index_col=0)
                            found=True
                        except:
                            try:
                                sheet1 = pd.read_excel(file_path,sheet_name=possible_sheet_names[5],index_col=0)
                                found=True
                            except:
                                try:
                                    sheet1 = pd.read_excel(file_path,sheet_name=possible_sheet_names[6],index_col=0)
                                    found=True
                                except:
                                    try:
                                        sheet1 = pd.read_excel(file_path,sheet_name=possible_sheet_names[7],index_col=0)
                                        found=True
                                    except:
                                        try:
                                            sheet1 = pd.read_excel(file_path,sheet_name=possible_sheet_names[8],index_col=0)
                                            found=True
                                        except:
                                            try:
                                                sheet1 = pd.read_excel(file_path,sheet_name=possible_sheet_names[9],index_col=0)
                                                found=True
                                            except:
                                                try:
                                                    sheet1 = pd.read_excel(file_path,sheet_name=possible_sheet_names[10],index_col=0)
                                                    found=True
                                                except:
                                                    pass
      
      if found==False:
        sheet1=pd.read_excel(file_path, sheet_name=3, index_col=0) 
           
      sheet1.index=sheet1.index.str.upper()


      temp1=get_key(sheet1.index.values)
      if temp1==0:
        try:
          sheet1=pd.read_excel(file_path,sheet_name=possible_sheet_names[11],index_col=0)
          sheet1.index=sheet1.index.str.upper()
          temp1=get_key(sheet1.index.values)
          if temp1==0:
            print(f"No key found for Ebitda of company {company_list[i]}")
            bad_companies.add(company_list[i])
            continue
            
        except:
          print(f"No key found for Ebitda of company {company_list[i]}")
          bad_companies.add(company_list[i])
          continue


      drop_cols=[]
      for col_name in sheet1.iloc[0].isnull().index.values:
        if sheet1.iloc[0].isnull()[col_name]:
          drop_cols.append(col_name)
      sheet1.drop(drop_cols, axis=1, inplace=True)


      sheet1.columns=sheet1.columns.str.upper()
      possible_cols=["1 MONTH ENDED", "1 MONTHS ENDED", "2 MONTHS ENDED", "3 MONTHS ENDED", "6 MONTHS ENDED", "42 MONTHS ENDED", 
                     "79 MONTHS ENDED", "40 MONTHS ENDED","9 MONTHS ENDED","83 MONTHS ENDED","5 MONTHS ENDED"]
      drop_cols=[c for c in possible_cols if c in sheet1.columns]
      sheet1.drop(drop_cols, axis=1, inplace=True)    


      index_name1=sheet1.index.name.split()
      if "Thousands" in index_name1 or "thousands" in index_name1:
        scale1="thousands"
      elif "Millions" in index_name1 or "millions" in index_name1:
        scale1="millions"
      

      
      
      years=len(sheet1.loc[temp1])
      df=sheet1.loc[temp1]

      if sheet1.loc[temp1].ndim>1:
        years=len(sheet1.loc[temp1].iloc[0])
        df=sheet1.loc[temp1].iloc[0]

      if years==2:
        col1=sheet1.iloc[0].values[0].split()[-1]+"-"+sheet1.iloc[0].values[1].split()[-1]
        ebitda_change.loc[company_list[i], col1]=(float(df[0])-float(df[1]))/100

        if scale1=="thousands":
          ebitda_change.loc[company_list[i], col1]=ebitda_change.loc[company_list[i], col1]*1000
        elif scale1=="millions":
          ebitda_change.loc[company_list[i], col1]=ebitda_change.loc[company_list[i], col1]*1000000
      
      elif years==3:
        col1=sheet1.iloc[0].values[1].split()[-1]+"-"+sheet1.iloc[0].values[2].split()[-1]
        ebitda_change.loc[company_list[i], col1]=(float(df[1])-float(df[2]))/100

        col=sheet1.iloc[0].values[0].split()[-1]+"-"+sheet1.iloc[0].values[1].split()[-1]
        ebitda_change.loc[company_list[i], col]=(float(df[0])-float(df[1]))/100

        if scale1=="thousands":
          ebitda_change.loc[company_list[i], col1]=ebitda_change.loc[company_list[i], col1]*1000
          ebitda_change.loc[company_list[i], col]=ebitda_change.loc[company_list[i], col]*1000
        elif scale1=="millions":
          ebitda_change.loc[company_list[i], col1]=ebitda_change.loc[company_list[i], col1]*1000000
          ebitda_change.loc[company_list[i], col]=ebitda_change.loc[company_list[i], col]*1000000

  except Exception as e:
    print(e)
    bad_companies.add(company_list[i])


In [ ]:
ebitda_change